In [ ]:
import pandas as pd

ratings_df = pd.read_csv("title_ratings_sample.tsv", sep="\t")
mpst_df = pd.read_csv("mpst_full_data.csv")

# Perform a left merge on the 'tconst' column from ratings_df and 'imdb_id' column from mpst_df
merged_df = ratings_df.merge(mpst_df, left_on="tconst", right_on="imdb_id", how="left")


In [ ]:
titles_df = pd.read_csv("title_basics_sample.tsv", sep="\t")

In [ ]:
titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short
...,...,...,...,...,...,...,...,...,...
999994,tt10058754,tvEpisode,Episode #1.59,Episode #1.59,0,2013,\N,\N,Family
999995,tt10058756,tvEpisode,Episode #1.60,Episode #1.60,0,2013,\N,\N,Family
999996,tt1005876,tvEpisode,Episode #1.1602,Episode #1.1602,0,1981,\N,22,"Drama,Romance"
999997,tt10058760,tvEpisode,Episode #1.339,Episode #1.339,0,2014,\N,\N,Family


In [ ]:
notna_merged_df = merged_df[merged_df['imdb_id'].notnull()]

In [ ]:
notna_merged_df

,tconst,averageRating,numVotes,imdb_id,title,plot_synopsis,tags,split,synopsis_source
89,tt0000091,6.7,4253,tt0000091,Le manoir du diable,The film opens with a large bat flying into a ...,"paranormal, gothic",train,wikipedia
207,tt0000225,5.1,40,tt0000225,La belle et la bête,A widower merchant lives in a mansion with his...,fantasy,train,wikipedia
210,tt0000230,6.5,1933,tt0000230,Cendrillon,"A prologue in front of the curtain, suppressed...",fantasy,train,wikipedia
300,tt0000417,8.1,58011,tt0000417,Le voyage dans la lune,"At a meeting of the Astronomic Club, its presi...","psychedelic, satire",train,imdb
330,tt0000488,6.9,927,tt0000488,The Land Beyond the Sunset,Joe is an impoverished New York newsboy who li...,"fantasy, storytelling",train,wikipedia
...,...,...,...,...,...,...,...,...,...
997954,tt2452254,6.7,31838,tt2452254,Clouds of Sils Maria,The opening scene shows a young woman named Va...,"boring, cruelty",test,imdb
998524,tt2458106,6.1,13749,tt2458106,Ninja: Shadow of a Tear,American martial artist Casey Bowman has settl...,revenge,train,wikipedia
998934,tt2461520,6.0,11411,tt2461520,Man Down,A U.S. Marine named Gabriel Drummer (Shia LaBe...,alternate reality,train,wikipedia
999348,tt2465238,6.4,5887,tt2465238,Lego Batman: The Movie - DC Super Heroes Unite,Bruce Wayne and Lex Luthor are both nominated ...,comedy,train,wikipedia


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.6 MB/s eta 0:00:00


In [ ]:
# Step 1: Load Data
movie_df = notna_merged_df.sample(5000, random_state=42)

In [ ]:
movie_df[movie_df['title'] == 'Hotel Transylvania']

,tconst,averageRating,numVotes,imdb_id,title,plot_synopsis,tags,split,synopsis_source


In [ ]:
import faiss
index_path = "movie_hnsw.index"
index = faiss.read_index(index_path)

In [ ]:
final_df = movie_df.merge(titles_df, left_on="imdb_id", right_on="tconst", how="left")

In [ ]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, util



# Keep only necessary columns and remove rows with missing plot synopses
movie_df = movie_df[['imdb_id', 'title', 'plot_synopsis']].dropna()

# Step 2: Generate SBERT Embeddings
print("Generating embeddings...")
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & fast
# plot_embeddings = model.encode(movie_df['plot_synopsis'].tolist(), batch_size=32, convert_to_tensor=False)


Generating embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
movie_df

,imdb_id,title,plot_synopsis
165625,tt0263975,Selvi Boylum Al Yazmalim,A classic and endlessly watchable love story t...
79919,tt0105869,Yao shou du shi,"The existence of the ""Black World"" - an altern..."
29354,tt0047863,Beanstalk Bunny,The story begins with Daffy Duck in the role o...
44918,tt0065569,"Nachts, wenn Dracula erwacht",The story starts with a shot of Count Dracula'...
407157,tt0805663,Jericho,"A World War I American troopship is torpedoed,..."
...,...,...,...
78700,tt0104403,Hedd Wyn,As the camera pans over the intricate carving ...
103261,tt0141369,Inspector Gadget,John Brown is a security guard working at the ...
238056,tt0418279,Transformers,"The movie opens with Optimus Prime, an Autobot..."
145282,tt0221073,Chopper,"In and out of jail since he was 16, Melbourne ..."


In [ ]:

# Convert to numpy array
plot_embeddings = np.array(plot_embeddings).astype('float32')

# Step 3: Build FAISS HNSW Index
embedding_dim = plot_embeddings.shape[1]
index = faiss.IndexHNSWFlat(embedding_dim, 32)
index.hnsw.efConstruction = 200

# Add embeddings to index
index.add(plot_embeddings)

# Save FAISS index
faiss.write_index(index, "movie_hnsw.index")
np.save("movie_ids.npy", movie_df['imdb_id'].values)

print("Index built and saved!")



Index built and saved!


In [ ]:
# Step 4: Define Movie Recommendation Function
def recommend_movies(query_title, top_k=10):
    """Finds the top-k similar movies based on plot synopsis."""
    # Find the query movie
    query_row = notna_merged_df[notna_merged_df['title'].str.lower() == query_title.lower()]
    if query_row.empty:
        return f"Movie '{query_title}' not found in database."

    query_embedding = model.encode(query_row.iloc[0]['plot_synopsis']).reshape(1, -1).astype('float32')

    # Search for similar movies
    distances, indices = index.search(query_embedding, top_k + 1)  # +1 to exclude the movie itself

    # Retrieve results
    recommendations = []
    for i, idx in enumerate(indices[0][1:]):  # Skip the first result (itself)
        movie_title = movie_df.iloc[idx]['title']
        recommendations.append((movie_title, distances[0][i+1]))

    return recommendations

In [ ]:

query_movie = "Hotel Transylvania"
recommendations = recommend_movies(query_movie, top_k=10)
print("Recommended Movies:", recommendations)


Recommended Movies: [('Captain Horatio Hornblower R.N.', 0.74598217), ('The Dunwich Horror', 0.85303336), ('And the Children Shall Lead', 0.89539623), ('Enid Is Sleeping', 0.90585554), ('The Adventures of Jim Bowie', 0.90817416), ('Michel Vaillant', 0.9284278), ('Vampire in Brooklyn', 0.93450934), ('Open Range', 0.9452752), ('Skullduggery', 0.9492428), ('Gears of War', 0.9529716)]
